# Network models

In this topic, you’ll learn what a network model is, and how its structure can be exploited for more efficient solutions.

## Networks in real life

Several problems encountered in Operations Research (OR) involve networks, such as:
Distribution problems (for example, transportation networks)
Assignment problems (for example, networks of workers and jobs they could be assigned to)
Planning problems (for example, critical path analysis for project planning)

Many network models are special LP problems for which specialized solution algorithms exist. 

It is important to know whether a problem can be formulated as a network model to exploit the special structure.

This topic introduces networks in general, as well as some well-known instances of network models.

## Network modeling concepts

Any network structure can be described using two types of objects:

- Nodes: Defined points in the network, for example warehouses.
- Arcs: An arc connects two nodes, for example a road connecting two warehouses. 

An arc can be _directed_, which means than an arc $a_{ij}$ from node $i$ to node $j$ is different from arc $a_ji$ that begins at node $j$ and ends at node $i$.


<img src = "https://www.tommasoadamo.it/images/lez19/1_5.png" >


 A sequence of arcs connecting two nodes is called a chain. Each arc in a chain shares exactly one node with the preceding arc.

 When all the arcs in a chain are directed such that it is possible to traverse the chain in the directions of the arcs from the start node to the end node, it is called a path.

<img src = "https://www.tommasoadamo.it/images/lez19/1_6.png" >
 

## Different types of network problems

The following are some well-known types of network problems:
- Transportation problem
- Assignment problem
- Shortest path problem

Next, you'll learn how to recognize each of these, and how their special structure can be exploited.

## The Transportation Problem

One of the most common real-world network problems is the transportation problem.  This type of problem involves a set of supply nodes $S$ and a set of demand nodes $T$.  The objective is to minimize the transportation cost $c_{ij}$ from the supply nodes to the demand nodes, so as to satisfy the demand $d_j$, and without exceeding the suppliers’ capacities $C_i$.  

Such a problem can be depicted in a graph, with supply nodes, demand nodes, and connecting arcs $A \subseteq S \times T$. The supply capacity is indicated with the supply nodes, while the demand is indicated with the demand nodes, and the transportation costs are indicated on the arcs.  

<img src = "https://www.tommasoadamo.it/images/lez19/1_8.png" />

<br>

<table style="font-size:15px">
    <tr>
        <td>
\begin{align}
\min\quad& \sum\limits_{i \in S} \sum\limits_{j \in T} c_{ij} x_{ij} & & \\
s.t.\quad& \sum\limits_{j \in T} x_{ij} \leq C_i & i \in S \\
\quad&\sum\limits_{i \in S} x_{ij} \geq d_j & j \in T \\
\quad&x_{ij}\geq 0& i \in S, j \in T
\end{align}
        </td>
        <td style="width: 100px"></td>
        <td>
\begin{align}
\min\quad& \sum\limits_{(i,j) \in A} c_{ij} x_{ij} & & \\
s.t.\quad& \sum\limits_{j \in T: \ (i,j) \in A} x_{ij} \leq C_i & i \in S \\
\quad&\sum\limits_{i \in S: \ (i,j) \in A} x_{ij} \geq d_j & j \in T \\
\quad&x_{ij}\geq 0& (i,j)\in A
\end{align}
        </td>
    </tr>
</table>

<br>

The LP formulation involves one type of variable, namely $x_{ij}$ representing the quantity transported from supply node $i$ to demand node $j$.  The objective is to minimize the total transportation cost across all arcs. The constraints are flow conservation constraints.  The first set of constraints state that the outflow from each supply node should be less than or equal to the supply capacity. The next set of constraints state that the inflow into each demand node should equal the demand at that node. The domain for the shipments on the allowable arcs is set to be greater than or equal to zero.  

Even though arcs (1,4) and (2,3) do not exist in the graph, the variables are included in the formulation to show the special structure of the transportation problem.  If you were to formulate such a model in practice, you’d simply exclude these variables. 

#### Formulating a simple transportation problem with DOcplex

In the next section, we formulate the problem described above using DOcplex.

#### What data for the transportation problem?

Input nodes are integers ranging in {1, 2}; output nodes are integers ranging from 3 to 5.

The data consists in three Python dictionaries:

- one dictionary gives capacity values for all input nodes
- one dictionary contains demands for all target nodes
- one last dictionary holds cost values for some (source, target) pair of nodes.

In [ ]:
source = (1, 2)
target = (3, 4, 5)
capacities = {1: 15, 2: 20}
demands = {3: 7, 4: 10, 5: 15}
costs = {(1,3): 2, (1,5): 4, (2,4): 5, (2,5): 3}

#### Create a model instance

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import docplex.mp.model as cplex

with cplex.Model('transportation') as tm:
    ...

#### Define the decision variables
- The continuous variables `x` represent the quantity transferred from supply nodes to demand nodes.

#### Set up the constraints

- For each source node, the total outbound flow must be smaller than available quantity.
- For each target node, total inbound flow must be greater than demand.

#### Express the business objective: minimize total flow cost

Each arc has a unit cost and we want to minimize the total cost. If an arc has no entry in the dictionary, we assume a $+\infty$ cost.

In [ ]:
import docplex.mp.model as cplex

c = {
    (1, 3): 2,
    (1, 5): 4,
    (2, 4): 5,
    (2, 5): 3
}

C = { 1: 15, 2: 20 }
d = { 3: 7, 4: 10, 5: 15 }

S = [ 1, 2 ]
T = [ 3, 4, 5]

with cplex.Model('problema del trasporto') as mdl:
    x = mdl.continuous_var_matrix(S, T, name='x')
    
    mdl.minimize(sum(c[i,j]*x[i,j] for i in S for j in T if (i,j) in c))
    
    for i in S:
        mdl.add_constraint(sum(x[i,j] for j in T if (i,j) in c) <= C[i], 'capacity'+str(i))
    
    for j in T:
        mdl.add_constraint(sum(x[i,j] for i in S if (i,j) in c) >= d[j], 'demand'+ str(j))
    
    
    mdl.print_information()
    
    s = mdl.solve()
    
    if s == None:
        print('No solution')
    else:
        print(s)
        for i,j in x:
            if x[i,j].solution_value > 0:
                print("Move {} from {} to {}".format(round(x[i,j].solution_value), i, j))

ALTERNATIVE WAY USING LISTS:

In [ ]:
import docplex.mp.model as cplex

S = (0, 1)
T = (0, 1, 2)
C = (15, 20)
d = (7, 10, 15)
c = [
    [2, 10000, 4],
    [10000, 5, 3]
]

with cplex.Model('transportation') as tm:
    # create flow variables for each couple of nodes
    # x(i,j) is the flow going out of node i to node j
    x = tm.continuous_var_matrix(S, T, name="x") 
    
    # for each node, total outgoing flow must be smaller than available quantity
    for i in S:
        tm.add_constraint(sum(x[i, j] for j in T) <= C[i], 'capacity_' + str(i))

    # for each target node, total ingoing flow must be greater thand demand
    for j in T:
        tm.add_constraint(sum(x[i, j] for i in S) >= d[j], 'demand_' + str(j))
    
    tm.minimize(sum(x[i,j]*c[i][j] for i in S for j in T))
    tm.print_information()
    tms = tm.solve()
    if tms != None:
        print(tms)
        for i,j in x:
            if x[i,j].solution_value > 0:
                print("Move {} from {} to {}".format(round(x[i,j].solution_value), i+1, j+3))
    else:
        print('No solution.')

### Solve with the Decision Optimization solve service

If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage may fail and will need a paying subscription or product installation.

In any case, `Model.solve()` returns a solution object in Python, containing the optimal values of decision variables, if the solve succeeds, or else it returns `None`.

### Special structure of  network problem

The special structure of the transportation problem, as well as many other types of network problems, allows the use of specialized algorithms that lead to significant reductions in solution time.

Another important characteristic of transportation problems (and also some other network problems) is that if all the capacities and demands are integer, then the decision variables will take integer values.

This is important to know, because it means that you do not have to use integer variables in such cases. 

As you'll learn in a later topic, integer variables often lead to problems that require much more computational effort compared to problems with only continuous variables.


## CPLEX Network Optimizer

As you’ve now seen, many network problems are special types of LP problems. In many cases, using the Simplex or Dual-simplex Optimizers is the most efficient way to solve them. In some cases, specialized algorithms can solve such problems more efficiently. 

CPLEX automatically invokes the Network Optimizer  when it's likely that it would improve solution time compared to the other algorithms. 

It is also possible to force the use (or not) of the Network Optimizer by setting the `lpopt` parameter of
a DOcplex model to 3 (remember 1 was primal simplex, 2 was dual simplex, and 4 is for barrier). 